<a href="https://colab.research.google.com/github/ishaqbreiwish/football-match-ml-predictor/blob/main/prediction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This notebook aims to predict football match outcomes using machine learning models, specifically using sklearn's random forest classifier. The dataset includes premier league matches from 2020 - 2022.

The data was preproccessed using pandas, to express non-integer values as integers, to make them valid inputs for the model. The precision of the model was also improved using rolling averages.

# Preprocessing the Data

The data first needs to be pre-processed using Pandas. First, we use the ID as the row index for easier traversal. Then we express the date, the venues, and the opponents into integers, as machine learnign models need integer inputs, and this information is all relevant to a team's probability of victory.

In [273]:
import pandas as pd

In [274]:
matches = pd.read_csv("matches.csv", index_col = 0) # Use the first column ('ID') as the row index instead of the default numeric index

In [275]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City


In [276]:
del matches["comp"]

In [277]:
del matches["notes"]

In [278]:
matches["date"] = pd.to_datetime(matches["date"]) # date is an object it needs to be an int/float for the model, so we convert it

In [279]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes #takes the venues, turns them into catgeories, and assigns a unique integer to each cat

In [280]:
matches["opp_code"] = matches["opponent"].astype("category").cat.codes #takes the opponents, turns them into catgeories, and assigns a unique integer to each cat

In [281]:
# isolating the time to just the hours
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int") #matches the colon and everything after it and replaces it w/ ""

In [282]:
matches["day_code"] = matches["date"].dt.dayofweek

In [283]:
matches["target"] = (matches["result"] == "W").astype("int")

In [284]:
matches

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
1,2021-08-15,16:30,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,1.9,...,1.0,0.0,0.0,2022,Manchester City,0,18,16,6,0
2,2021-08-21,15:00,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,2.7,...,1.0,0.0,0.0,2022,Manchester City,1,15,15,5,1
3,2021-08-28,12:30,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,3.8,...,0.0,0.0,0.0,2022,Manchester City,1,0,12,5,1
4,2021-09-11,15:00,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,2.9,...,0.0,0.0,0.0,2022,Manchester City,0,10,15,5,1
6,2021-09-18,15:00,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,1.1,...,1.0,0.0,0.0,2022,Manchester City,1,17,15,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2021-05-02,19:15,Matchweek 34,Sun,Away,L,0.0,4.0,Tottenham,0.5,...,0.0,0.0,0.0,2021,Sheffield United,0,18,19,6,0
39,2021-05-08,15:00,Matchweek 35,Sat,Home,L,0.0,2.0,Crystal Palace,0.7,...,1.0,0.0,0.0,2021,Sheffield United,1,6,15,5,0
40,2021-05-16,19:00,Matchweek 36,Sun,Away,W,1.0,0.0,Everton,1.6,...,0.0,0.0,0.0,2021,Sheffield United,0,7,19,6,1
41,2021-05-19,18:00,Matchweek 37,Wed,Away,L,0.0,1.0,Newcastle Utd,0.8,...,1.0,0.0,0.0,2021,Sheffield United,0,14,18,2,0


# Initializing the Machine Learning Model

We initialize the random forest classifier from sklearn, which we will be training on our data. Then we initialize a set of training data and a set of test data. We set our predictors, the venue, the opponent, the time and date.

In [285]:
from sklearn.ensemble import RandomForestClassifier

In [286]:
rf = RandomForestClassifier(n_estimators=40, min_samples_split=10, random_state = 1) #initializes random forest

In [287]:
train = matches[matches["date"] < '2022-01-01'] #initializing training data on history of matches

In [288]:
test = matches[matches["date"] > '2022-01-01'] # initializing test data on current matches

In [289]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [290]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=40, random_state=1)

In [291]:
preds = rf.predict(test[predictors])

# Accuracy & Precision

We check the accuracy and precision of our model using sklearn.metrics. Then, to improve precision, we calcualte rolling averages based on the previous 3 maches, as recent performance can be a good indicator of future performance

In [292]:
from sklearn.metrics import accuracy_score

In [293]:
acc = accuracy_score(test["target"], preds)

In [294]:
acc

0.6195652173913043

In [295]:
combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))

In [296]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,146,26
1,79,25


In [297]:
from sklearn.metrics import precision_score

In [298]:
precision_score(test["target"], preds)

0.49019607843137253

In [299]:
grouped_matches = matches.groupby("team") # group matches by a certain team

In [300]:
group = grouped_matches.get_group("Manchester City").sort_values("date")

In [301]:
def rolling_averages(group, cols, new_cols):
  group = group.sort_values("date")
  rolling_stats = group[cols].rolling(3, closed="left").mean() #closed = left makes sure the rolling avg. is before current week to make sure we dont include it
  group[new_cols] = rolling_stats
  group = group.dropna(subset=new_cols) #drops missing values rolled in when there is less than 3 previous matches, most ml models cant handle missing values
  return group

In [302]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols] # just creates new cols w/ all those variables but rolling after

In [303]:
rolling_averages(group, cols, new_cols)

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
5,2020-10-17,17:30,Matchweek 5,Sat,Home,W,1.0,0.0,Arsenal,1.5,...,5,1,2.000000,2.333333,17.333333,4.666667,18.900000,1.333333,0.333333,0.333333
7,2020-10-24,12:30,Matchweek 6,Sat,Away,D,1.0,1.0,West Ham,1.1,...,5,0,1.333333,2.000000,17.333333,3.666667,17.733333,0.666667,0.000000,0.000000
9,2020-10-31,12:30,Matchweek 7,Sat,Away,W,1.0,0.0,Sheffield Utd,1.5,...,5,1,1.000000,0.666667,16.666667,4.333333,18.233333,0.666667,0.000000,0.000000
11,2020-11-08,16:30,Matchweek 8,Sun,Home,D,1.0,1.0,Liverpool,1.6,...,6,0,1.000000,0.333333,14.333333,6.666667,18.466667,1.000000,0.000000,0.000000
12,2020-11-21,17:30,Matchweek 9,Sat,Away,L,0.0,2.0,Tottenham,1.3,...,5,0,1.000000,0.666667,12.000000,5.666667,19.366667,1.000000,0.000000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,2022-03-14,20:00,Matchweek 29,Mon,Away,D,0.0,0.0,Crystal Palace,2.3,...,0,0,2.333333,1.333333,19.000000,7.000000,15.366667,0.333333,0.333333,0.333333
44,2022-04-02,15:00,Matchweek 31,Sat,Away,W,2.0,0.0,Burnley,1.8,...,5,1,1.666667,0.333333,18.333333,7.333333,16.000000,0.333333,0.000000,0.000000
46,2022-04-10,16:30,Matchweek 32,Sun,Home,D,2.0,2.0,Liverpool,2.0,...,6,0,2.000000,0.333333,20.000000,6.666667,16.133333,0.333333,0.000000,0.000000
49,2022-04-20,20:00,Matchweek 30,Wed,Home,W,3.0,0.0,Brighton,1.2,...,2,1,1.333333,0.666667,15.666667,4.666667,16.700000,0.333333,0.000000,0.000000


In [304]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

<ipython-input-304-d7a196cb9ce7>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))


In [305]:
matches_rolling

date   time         round  day venue result  \
team                                                                           
Arsenal                 6  2020-10-04  14:00   Matchweek 4  Sun  Home      W   
                        7  2020-10-17  17:30   Matchweek 5  Sat  Away      L   
                        9  2020-10-25  19:15   Matchweek 6  Sun  Home      L   
                        11 2020-11-01  16:30   Matchweek 7  Sun  Away      W   
                        13 2020-11-08  19:15   Matchweek 8  Sun  Home      L   
...                               ...    ...           ...  ...   ...    ...   
Wolverhampton Wanderers 32 2022-03-13  14:00  Matchweek 29  Sun  Away      W   
                        33 2022-03-18  20:00  Matchweek 30  Fri  Home      L   
                        34 2022-04-02  15:00  Matchweek 31  Sat  Home      W   
                        35 2022-04-08  20:00  Matchweek 32  Fri  Away      L   
                        36 2022-04-24  14:00  Matchweek 34  Sun  Away      L   

                             gf   ga         opponent   xg  ...  day_code  \
team                                                        ...             
Arsenal                 6   2.0  1.0    Sheffield Utd  0.4  ...         6   
                        7   0.0  1.0  Manchester City  0.9  ...         5   
                        9   0.0  1.0   Leicester City  0.9  ...         6   
                        11  1.0  0.0   Manchester Utd  1.1  ...         6   
                        13  0.0  3.0      Aston Villa  1.5  ...         6   
...                         ...  ...              ...  ...  ...       ...   
Wolverhampton Wanderers 32  1.0  0.0          Everton  0.8  ...         6   
                        33  2.0  3.0     Leeds United  0.8  ...         4   
                        34  2.0  1.0      Aston Villa  1.2  ...         5   
                        35  0.0  1.0    Newcastle Utd  0.3  ...         4   
                        36  0.0  1.0          Burnley  0.7  ...         6   

                            target  gf_rolling ga_rolling sh_rolling  \
team                                                                   
Arsenal                 6        1    2.000000   1.333333   7.666667   
                        7        0    1.666667   1.666667   5.333333   
                        9        0    1.000000   1.666667   7.000000   
                        11       1    0.666667   1.000000   9.666667   
                        13       0    0.333333   0.666667   9.666667   
...                            ...         ...        ...        ...   
Wolverhampton Wanderers 32       1    1.333333   1.000000  12.333333   
                        33       0    1.666667   0.666667  12.333333   
                        34       1    2.333333   1.000000  13.000000   
                        35       0    1.666667   1.333333  13.000000   
                        36       0    1.333333   1.666667  10.000000   

                           sot_rolling dist_rolling  fk_rolling  pk_rolling  \
team                                                                          
Arsenal                 6     3.666667    14.733333    0.666667    0.000000   
                        7     3.666667    15.766667    0.000000    0.000000   
                        9     3.666667    16.733333    0.666667    0.000000   
                        11    4.000000    16.033333    1.000000    0.000000   
                        13    2.666667    18.033333    1.000000    0.333333   
...                                ...          ...         ...         ...   
Wolverhampton Wanderers 32    3.666667    19.300000    0.000000    0.000000   
                        33    4.333333    19.600000    0.000000    0.000000   
                        34    5.333333    19.833333    0.000000    0.000000   
                        35    5.000000    18.533333    0.000000    0.000000   
                        36    4.666667    17.633333    0.000000    0.000000   

                         

In [306]:
matches_rolling = matches_rolling.droplevel('team') # dropping the uneccessary new team column

In [307]:
matches_rolling.index = range(matches_rolling.shape[0]) # makes it such that we have unique values for every index

# Retraining the model

We make new predictions based on the new data (the rolling averages). Additionally, when a team is listed as an opponent vs as the main team, their names are sometimes different so we override the missing method for the dictionary to be able to replace all these values in our table.

In [308]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2022-01-01']
    test = data[data["date"] > '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision

In [309]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [310]:
precision

0.625

In [266]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)
# merges date/ team info to make it more clear where the model isn't working + but makes sure column matches row

In [267]:
class MissingDict(dict):  #by default, pandas map method doesn't handle missing keys
  __missing__ = lambda self, key: key #missing method is overridden to instead just return the same key when you get a missing key error

map_values = { #done because team names are written differently on the home/away sides
    "Brighton and Hove Albion" : "Brighton",
    "Manchester United": "Manchester Utd",
    "Newcastle United" : "Newcastle Utd",
    "Tottenham Hotspur" : "Tottenham",
    "Westham United" : "West Ham",
    "Wolverhampton Wanderers" : "Wolves"
}

mapping = MissingDict(**map_values)

In [268]:
combined['new_team'] = combined["team"].map(mapping) # replaces each team name with its value in map_values

In [269]:
merged = combined.merge(combined, left_on=["date","new_team"], right_on=["date", "opponent"])

In [270]:
merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,0,0,2022-01-23,Arsenal,Burnley,D,Arsenal,0,0,Burnley,Arsenal,D,Burnley
1,1,0,2022-02-10,Arsenal,Wolves,W,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,Wolves
2,1,0,2022-02-19,Arsenal,Brentford,W,Arsenal,0,0,Brentford,Arsenal,L,Brentford
3,1,1,2022-02-24,Arsenal,Wolves,W,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,Wolves
4,1,1,2022-03-06,Arsenal,Watford,W,Arsenal,0,0,Watford,Arsenal,L,Watford
...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,1,0,2022-03-13,Wolverhampton Wanderers,Everton,W,Wolves,0,0,Everton,Wolves,L,Everton
245,0,0,2022-03-18,Wolverhampton Wanderers,Leeds United,L,Wolves,1,0,Leeds United,Wolves,W,Leeds United
246,1,0,2022-04-02,Wolverhampton Wanderers,Aston Villa,W,Wolves,0,0,Aston Villa,Wolves,L,Aston Villa
247,0,0,2022-04-08,Wolverhampton Wanderers,Newcastle Utd,L,Wolves,1,0,Newcastle United,Wolves,W,Newcastle Utd


In [271]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] ==0)]["actual_x"].value_counts()

,count
actual_x,
1,25
0,13


In [272]:
precision_score

<function sklearn.metrics._classification.precision_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')>